In [1]:
from Initialization import initialize_Ising
from tqdm import tqdm
import numpy as np
import cft
import loop_opt as tnr

In [2]:
def auto_calculator(T,d_cut):
    qr = True
    size=24
    algo = 'loop'
    if algo == 'loop':
        opt_error = 1.0E-14
        maxloop = 100
        algo = 'loop'
    elif algo == 'ln':
        opt_error = np.inf
        maxloop = -1
        algo = 'ln'

    if size % 2 != 0:
        sys.exit("System size should be an even number")

    field = 0.0
    temperature=T
    beta=1./temperature
    keep_num = d_cut ** 2
    cft_result = np.zeros((size, keep_num))
    error_list=np.zeros((size))

    # assign the initial value of tensor TA, TB

    ts_A0 = initialize_Ising(temperature)
    ts_B0 = ts_A0.copy()
    # normalize tensor before RG
    ts_A0, ts_B0, plqnorm = cft.plq_normalize_new(ts_A0, ts_B0)
    # initial value ln(z_0)
    area = 4
    plqnorm_list=[plqnorm]
    ln_z = np.log(plqnorm)/area
    # initial area per tensor = 2*2 because of the normalization

    for i in tqdm(range(size)):
        ts_A0, ts_B0 = flt.filter(ts_A0, ts_B0)
    # loop optimize to find the new tensors
        ts_A0, ts_B0, loop_error, ln_error = opt.loop_opt(ts_A0, ts_B0, d_cut, opt_error, maxloop, qr=qr, return_LN_error=True)
        error_list[i]=loop_error
    # normalize the new tensors in the plaquette
    # construct the 2 x 2 transfer matrix
        ts_A0, ts_B0, plqnorm = cft.plq_normalize_new(ts_A0, ts_B0)
        area *= 2
        ln_z += np.log(plqnorm) / area
        plqnorm_list.append(plqnorm)
    # calculate central charge and scaling dimension
        data = cft.cal_sp_new(ts_A0, ts_B0, plqnorm)
        cft_result[i, 0:min(keep_num, np.shape(data)[0])] = data[0:min(keep_num, np.shape(data)[0])]
    np.save("tnr_data/Critical_{}.npy".format(d_cut),cft_result)
    np.save("tnr_/plqnorm_{}.npy".format(d_cut),plqnorm_list)
    return 0